In [1]:
!docker pull nvcr.io/nvidia/tritonserver:21.08-py3

21.08-py3: Pulling from nvidia/tritonserver
Digest: sha256:6a1ef84e93327fef53ac11379fa4e53082c9e228de5db24e0abdfc5aff5676cf
Status: Image is up to date for nvcr.io/nvidia/tritonserver:21.08-py3
nvcr.io/nvidia/tritonserver:21.08-py3


In [28]:
!rm -rf triton_models
!mkdir -p triton_models
!mkdir -p triton_models/resnet1/
!mkdir -p triton_models/resnet1/1/
!cp traced_resnet_model.pt triton_models/resnet1/1/model.pt

!mkdir -p triton_models/resnet2/
!mkdir -p triton_models/resnet2/1/
!cp traced_resnet_model.pt triton_models/resnet2/1/model.pt


In [29]:
%%writefile triton_models/resnet1/config.pbtxt  
name: "resnet1"
platform: "pytorch_libtorch"
input [
 {
    name: "input__0"
    data_type: TYPE_FP32 
    dims: [-1, 3, 224, 224]
  } 
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [1000]
  }

instance_group [
  {
      kind:KIND_GPU
  } 
]

Writing triton_models/resnet1/config.pbtxt


In [30]:
%%writefile triton_models/resnet2/config.pbtxt  
name: "resnet2"
platform: "pytorch_libtorch"
input [
 {
    name: "input__0"
    data_type: TYPE_FP32 
    dims: [-1, 3, 224, 224]
  } 
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [1000]
  }

instance_group [
  {
      kind:KIND_GPU
  } 
]

Writing triton_models/resnet2/config.pbtxt


In [32]:
!docker run --gpus=1 --rm -p8000:8000 -p8001:8001 -p8002:8002 -v $PWD/triton_models:/models 3eb9530b33c0 tritonserver --model-repository=/models


== Triton Inference Server ==

NVIDIA Release 21.08 (build 26170506)

Copyright (c) 2018-2021, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.

This container image and its contents are governed by the NVIDIA Deep Learning Container License.
By pulling and using the container, you accept the terms and conditions of this license:
https://developer.nvidia.com/ngc/nvidia-deep-learning-container-license

NOTE: Legacy NVIDIA Driver detected.  Compatibility mode ENABLED.

I0922 14:35:40.258040 1 metrics.cc:290] Collecting metrics for GPU 0: Tesla V100-SXM2-16GB
I0922 14:35:40.592443 1 libtorch.cc:1029] TRITONBACKEND_Initialize: pytorch
I0922 14:35:40.592483 1 libtorch.cc:1039] Triton TRITONBACKEND API version: 1.4
I0922 14:35:40.592490 1 libtorch.cc:1045] 'pytorch' TRITONBACKEND API version: 1.4
2021-09-22 14:35:40.757752: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully ope

I0922 14:45:38.887582 1 libtorch.cc:1101] TRITONBACKEND_ModelFinalize: delete model state
I0922 14:45:38.887592 1 libtorch.cc:1101] TRITONBACKEND_ModelFinalize: delete model state
I0922 14:45:38.887924 1 model_repository_manager.cc:1195] successfully unloaded 'resnet1' version 1
I0922 14:45:38.887942 1 model_repository_manager.cc:1195] successfully unloaded 'resnet2' version 1
I0922 14:45:39.880425 1 server.cc:249] Timeout 29: Found 0 live models and 0 in-flight non-inference requests
